In [1]:
# Step 1 — Install required libraries
!pip install ultralytics



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 75.9 MB/s eta 0:00:00


In [13]:
# Step 1: Setup and unzip dataset
from pathlib import Path
import zipfile

# Path to uploaded dataset (adjust if different)
zip_path = "/content/archive.zip"
root_dir = Path("/content/data")

# Unzip dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(root_dir)

print(" Dataset extracted to:", root_dir)


 Dataset extracted to: /content/data


In [15]:
# Step 2: Split data into train/val folders (80/20)
import os, random, shutil
from pathlib import Path

source_train = Path("/content/classification_data_fixed/train")
target_val = Path("/content/classification_data_fixed/val")

# Create val folders if missing
for cls in [d.name for d in source_train.iterdir() if d.is_dir()]:
    os.makedirs(target_val / cls, exist_ok=True)

val_split = 0.2
for cls in [d.name for d in source_train.iterdir() if d.is_dir()]:
    img_files = list((source_train / cls).glob("*"))
    val_count = int(len(img_files) * val_split)
    val_samples = random.sample(img_files, val_count)
    for img in val_samples:
        shutil.move(str(img), target_val / cls / img.name)

print(" Split complete!")
for cls in [d.name for d in source_train.iterdir() if d.is_dir()]:
    print(f"{cls}: Train = {len(list((source_train/cls).glob('*')))}, Val = {len(list((target_val/cls).glob('*')))}")


 Split complete!
Breeding Place: Train = 461, Val = 439
Mosquito Swarm: Train = 18, Val = 15
Mosquito: Train = 62, Val = 58


In [16]:
# Step 3: Load image datasets for training
from tensorflow.keras.preprocessing import image_dataset_from_directory
from pathlib import Path

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
output_root = Path("/content/classification_data_fixed")

train_ds = image_dataset_from_directory(
    output_root / "train",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

# Handle empty val/test folders safely
val_dir = output_root / "val"
if any(val_dir.glob("*/*")):
    val_ds = image_dataset_from_directory(val_dir, image_size=IMG_SIZE, batch_size=BATCH_SIZE, label_mode='categorical')
else:
    print("⚠️ Validation directory empty — skipping val_ds.")
    val_ds = None

test_dir = output_root / "test"
if any(test_dir.glob("*/*")):
    test_ds = image_dataset_from_directory(test_dir, image_size=IMG_SIZE, batch_size=BATCH_SIZE, label_mode='categorical')
else:
    print("⚠️ Test directory empty — skipping test_ds.")
    test_ds = None

class_names = train_ds.class_names
num_classes = len(class_names)
print("✅ Remaining classes for training:", class_names)
print(f"📊 Number of classes: {num_classes}")


Found 541 files belonging to 3 classes.
Found 512 files belonging to 3 classes.
Found 51 files belonging to 3 classes.
✅ Remaining classes for training: ['Breeding Place', 'Mosquito', 'Mosquito Swarm']
📊 Number of classes: 3


In [17]:
# Step 4: Optimize data pipeline
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
if val_ds:
    val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [18]:
# Step 5: Build and compile EfficientNet model
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0

base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base for feature extraction

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,213,926 (16.07 MB)

 Trainable params: 164,355 (642.01 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [19]:
# Step 6: Train the model
EPOCHS = 10

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)


Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.8974 - loss: 0.3635 - val_accuracy: 0.9629 - val_loss: 0.1419
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.9776 - loss: 0.0681 - val_accuracy: 0.9570 - val_loss: 0.1151
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.9946 - loss: 0.0282 - val_accuracy: 0.9727 - val_loss: 0.1074
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - accuracy: 0.9979 - loss: 0.0147 - val_accuracy: 0.9746 - val_loss: 0.0969
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - accuracy: 0.9983 - loss: 0.0089 - val_accuracy: 0.9688 - val_loss: 0.0886
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - accuracy: 1.0000 - loss: 0.0070 - val_accuracy: 0.9746 - val_loss: 0.0857
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 1.0000 - loss: 0.0051 - val_accuracy: 0.9746 - val_loss: 0.0856
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 1.0000 - loss: 0.0057 - val_accuracy: 0.9746 - va

In [21]:
# Save in both formats
model.save("/content/mosquito_risk_model.keras")  # ✅ Modern format
model.save("/content/mosquito_risk_model.h5")     # 🧩 Legacy format

print("Model saved in both formats: .keras and .h5")


Model saved in both formats: .keras and .h5


In [22]:
# Modern version (recommended)
model = tf.keras.models.load_model("/content/mosquito_risk_model.keras")

# OR if needed for older environments
# model = tf.keras.models.load_model("/content/mosquito_risk_model.h5")


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 6 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [23]:
# Step 8: Predict mosquito risk for any image
from tensorflow.keras.preprocessing import image
import numpy as np

# Define custom risk mapping
risk_mapping = {
    "Breeding Place": "High Risk",
    "Mosquito": "Moderate Risk",
    "Mosquito Swarm": "High Risk"
}

def predict_risk(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    preds = model.predict(img_array)
    class_idx = np.argmax(preds)
    class_name = class_names[class_idx]
    confidence = preds[0][class_idx] * 100
    risk_level = risk_mapping.get(class_name, "Unknown")

    print(f"🦟 Detected: {class_name}")
    print(f"🔥 Risk Level: {risk_level}")
    print(f"🎯 Confidence: {confidence:.2f}%")

# Example — upload and test your own image
# from google.colab import files
# uploaded = files.upload()
# predict_risk(list(uploaded.keys())[0])


In [24]:
from tensorflow.keras.models import load_model

# Load your saved model (.keras or .h5)
model = load_model("/content/mosquito_risk_model.keras", compile=False)


In [25]:
from google.colab import files
uploaded = files.upload()


Saving img1.jpg to img1.jpg


In [26]:
predict_risk(list(uploaded.keys())[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step
🦟 Detected: Mosquito
🔥 Risk Level: Moderate Risk
🎯 Confidence: 86.47%


In [27]:
# 🔹 Load model
from tensorflow.keras.models import load_model
model = load_model("/content/mosquito_risk_model.keras", compile=False)

# 🔹 Upload and predict
from google.colab import files
uploaded = files.upload()

for fname in uploaded.keys():
    print(f"\n📸 Predicting for: {fname}")
    predict_risk(fname)


Saving img1.jpg to img1 (1).jpg
Saving img2.jpg to img2.jpg
Saving img3.jpg to img3.jpg
Saving img3.webp to img3.webp
Saving img4.jpg to img4.jpg

📸 Predicting for: img1 (1).jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
🦟 Detected: Mosquito
🔥 Risk Level: Moderate Risk
🎯 Confidence: 86.47%

📸 Predicting for: img2.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
🦟 Detected: Mosquito
🔥 Risk Level: Moderate Risk
🎯 Confidence: 86.47%

📸 Predicting for: img3.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
🦟 Detected: Mosquito
🔥 Risk Level: Moderate Risk
🎯 Confidence: 86.50%

📸 Predicting for: img3.webp
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
🦟 Detected: Mosquito
🔥 Risk Level: Moderate Risk
🎯 Confidence: 86.48%

📸 Predicting for: img4.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
🦟 Detected: Mosquito
🔥 Risk Level: Moderate Risk
🎯 Confidence: 86.47%
